In [1]:
import os
from django.conf import settings
from feed import models as feed_models
from accounts import models
from django.db.models import BooleanField
from django.db.models.expressions import Q, F, Case, When

In [2]:
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

In [39]:
users = models.MyUser.objects.all()
user1 = users.first()
user2 = users.get(username='kylie')

In [40]:
profile1 = user1.myuserprofile
profile2 = user2.myuserprofile
profile2

<MyUserProfile: kylie.jenner@gmail.com>

In [41]:
followers = profile1.followed_by.all()
followers

<QuerySet [<MyUserProfile: kylie.jenner@gmail.com>]>

In [42]:
profile2 in followers

True

In [44]:
juliette = users.get(username='juliette')
# constance.myuserprofile.follows.add(profile1)
juliette.myuserprofile.follows.all()

<QuerySet [<MyUserProfile: kylie.jenner@gmail.com>]>

In [46]:
# constance.myuserprofile.follows.remove(profile1)
juliette.myuserprofile.follows.all()

<QuerySet [<MyUserProfile: kylie.jenner@gmail.com>]>

In [3]:
videos = feed_models.Video.objects.all()

In [8]:
video = videos.first()

## Likes

In [77]:
likes = videos.like_set.all()
print(likes, likes.count())

<QuerySet [<Like: kylie>, <Like: juliette>, <Like: zadigo>]> 3


In [81]:
from django.db.models import Count

juliette_videos = juliette.video_set.all()
juliette_videos.aggregate(Count('like'))

# feed_models.Like.objects.filter(user__username='zadigo').count()

5

## Check user has liked video

In [13]:
user_in_likes = When(like__user__username='zadigo', then=True)
cases = Case(user_in_likes, default=False, output_field=BooleanField())
annotated_comments = comments.annotate(has_liked=cases)
annotated_comments.values_list('has_liked')

<QuerySet [(False,), (False,), (True,), (False,), (False,), (True,), (True,), (False,), (False,)]>

In [14]:
zadigo = users.get(username='zadigo')

# Tags

In [3]:
from tags import models

In [4]:
tag = models.Tag.objects.first()

In [6]:
tag.video_set.filter(public=True)

<QuerySet [<Video: 3d102d3d9f>]>

# Hashatgs and references

In [17]:
from string import Template
import re
from django.utils.html import mark_safe
from nltk.tokenize import NLTKWordTokenizer, TweetTokenizer

def hashtags(text):
    new_tokens = []
    tag = Template("""<a href="$href">$name</a>""")

    instance = TweetTokenizer()
    tokens = instance.tokenize(text)
    print(tokens)
    for token in tokens:
        if token.startswith('#'):
            value = re.match(r'\#(\w+)', token)
            if value:
                href = reverse('tags:tag', args=[value.group(1)])
                token = tag.substitute(href=href, name=token)
        new_tokens.append(token)
    return mark_safe(' '.join(new_tokens))

hashtags("I did it . Now what ? #fyp #TheWeekndEXP #hairdye #blackhair #alt")

['I', 'did', 'it', '.', 'Now', 'what', '?', '#fyp', '#TheWeekndEXP', '#hairdye', '#blackhair', '#alt']


'I did it . Now what ? <a href="/tags/fyp">#fyp</a> <a href="/tags/TheWeekndEXP">#TheWeekndEXP</a> <a href="/tags/hairdye">#hairdye</a> <a href="/tags/blackhair">#blackhair</a> <a href="/tags/alt">#alt</a>'

# Suggested users

In [24]:
follows_ids = list(models.MyUser.objects.first().myuserprofile.follows.values_list('id', flat=True))

In [26]:
models.MyUserProfile.objects.exclude(id__in=follows_ids)

<QuerySet [<MyUserProfile: pendenquejohn@gmail.com>, <MyUserProfile: juliettebinocha@gmail.com>]>

# Search

In [36]:
q = "drafts"
logic = (
    Q(caption__icontains=q) |
    Q(user__username__icontains=q) |
    Q(tags__title__icontains=q)
)
feed_models.Video.objects.filter(logic).distinct()

<QuerySet [<Video: 7d6c06d5dc>, <Video: 71c477ea50>]>

In [11]:
models.MyUser.objects.filter(username__in=['zadigo'])

<QuerySet [<MyUser: pendenquejohn@gmail.com>]>